# Video 

-  5.4.2 - GroupBy in Spark
-  [![](https://markdown-videos-api.jorgenkh.no/youtube/9qrDsY_2COo)](https://www.youtube.com/watch?v=9qrDsY_2COo&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=60)

We've been using local spark clusters with below code.

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
	.master("local[*]") \
	.appName('test') \
	.getOrCreate()
	
spark.sparkContext.setLogLevel("ERROR")

your 131072x1 screen size is bogus. expect trouble
24/03/02 00:24:03 WARN Utils: Your hostname, Cinders resolves to a loopback address: 127.0.1.1; using 172.17.156.62 instead (on interface eth0)
24/03/02 00:24:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 00:24:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read parquet `green`

In [2]:
df_green = spark.read.parquet('../../data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

### Changed from grouping by monthly to hourly

filtered for rows starting from '2020-01-01 00:00:00'

In [8]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2

""")

```sql
ORDER BY
    1, 2
```

removed

In [9]:
df_green_revenue.show(5)

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-09 17:00:00| 225|             82.46|             5|
|2020-01-02 21:00:00|  74| 506.4200000000001|            38|
|2020-01-27 13:00:00|  24|            203.57|             7|
|2020-01-06 16:00:00|  43|343.68000000000006|            21|
|2020-01-05 08:00:00|  55|132.51999999999998|             4|
+-------------------+----+------------------+--------------+
only showing top 5 rows



In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('../../data/report/revenue/green', mode='overwrite')

## Read parquet yellow

In [11]:
df_yellow = spark.read.parquet('../../data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [13]:
df_yellow_revenue.show(5)

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-25 12:00:00| 162| 7347.189999999992|           488|
|2020-01-21 22:00:00| 164| 4912.309999999998|           268|
|2020-01-24 20:00:00| 186| 7087.639999999995|           398|
|2020-01-18 12:00:00| 107| 3927.919999999998|           252|
|2020-01-19 13:00:00| 164|3883.5399999999986|           224|
+-------------------+----+------------------+--------------+
only showing top 5 rows



In [14]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../../data/report/revenue/yellow', mode='overwrite')

## Read report revenue

we read back the revenue reports into spark dataframe

video: 5.4.3 Joins in Spark

[![](https://markdown-videos-api.jorgenkh.no/youtube/lu7TrqAWuH4)](https://youtu.be/lu7TrqAWuH4&list=PL3MmuxUbc_hJed7dXYoJw8DoCuVHhGEQb&index=60)

In [15]:
df_green_revenue = spark.read.parquet('../../data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../../data/report/revenue/yellow')

In [16]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

## join green & yellow

In [17]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [18]:
df_join.show(5)

+-------------------+----+------------------+--------------------+-----------------+---------------------+
|               hour|zone|      green_amount|green_number_records|    yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10|           409.35|                   19|
|2020-01-01 00:00:00|  68|              NULL|                NULL|7825.069999999994|                  396|
|2020-01-01 00:00:00|  88|              NULL|                NULL|823.8000000000001|                   36|
|2020-01-01 00:00:00|  93|              NULL|                NULL|           210.28|                    3|
|2020-01-01 00:00:00| 106|             10.56|                   1|             NULL|                 NULL|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
only showing top 5 rows



## write `df_join` to revenue/report/total

In [14]:
df_join.write.parquet('../../data/report/revenue/total', mode='overwrite')

## read back parquet from revenue/report/total

In [15]:
df_join = spark.read.parquet('../../data/report/revenue/total')

In [16]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

## read parquet zones

In [19]:
df_zones = spark.read.parquet('../../data/zones/')

In [23]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



## join `df_join` & `df_zones`

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [22]:
df_join.show(5)

+-------------------+----+------------------+--------------------+-----------------+---------------------+
|               hour|zone|      green_amount|green_number_records|    yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10|           409.35|                   19|
|2020-01-01 00:00:00|  68|              NULL|                NULL|7825.069999999994|                  396|
|2020-01-01 00:00:00|  88|              NULL|                NULL|823.8000000000001|                   36|
|2020-01-01 00:00:00|  93|              NULL|                NULL|           210.28|                    3|
|2020-01-01 00:00:00| 106|             10.56|                   1|             NULL|                 NULL|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
only showing top 5 rows



In [24]:
df_result.drop('LocationID', 'zone').write.parquet('../../data/tmp/revenue-zones', mode="ignore")